In [141]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
import math
cv2.startWindowThread()


0

In [142]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Daniel_7_top_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps
print(fps)
if cap.isOpened() == False:
    print("Cannot open video file.")

119.88011988011988


In [143]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)


In [144]:
#defining size for output videos
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1920,1080))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [145]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [146]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])
prev_x, prev_y = None, None
prev_time = None


In [147]:
#defining rgb range to detect orange for now
lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])

# blue_lower_bound = np.array([60,50,0])
# blue_upper_bound = np.array([170,255,255])

In [148]:
#conversion factor, likely incorrect bc camera and setup need to be calibrated
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess
conversion_rate = 0
rect_coordinates = 0
new_data = None

In [149]:
orb = cv2.ORB_create()

In [150]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    
    height, width, _ = frame.shape
    
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #trying rgb threshold
    mask = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    
    longest_width = 0
    for cnt in rgb_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if w > longest_width:
                longest_width = w
            #print(longest_width)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0),3)
        
        # keypoints, descriptors = orb.detectAndCompute(cnt, None)
        # frame_keypoints = cv2.drawKeypoints(frame, keypoints, None)
            
    if longest_width > 0:
        conversion_rate = disc_length / longest_width
    

    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = (dx / dt) 
        vy = (dy / dt)
        
        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate
        vy_mph = vy * conversion_rate
        velocity = np.linalg.norm([vx_mph, vy_mph])

        # Display the velocity values on the output video
        cv2.putText(frame, 'X Velocity: {:.2f} mph'.format(vx_mph), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'Y Velocity: {:.2f} mph'.format(vy_mph), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'True Velocity: {:.2f} mph'.format(velocity), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})

    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time

    cv2.imshow("frame", frame)
    # cv2.imshow("frame keypoints", frame_keypoints)
    #cv2.imshow("roi", roi)
    #cv2.imshow("Mask", mask)
    #cv2.imshow("maskrgb",mask_rgb)
    
    output.write(frame)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

In [151]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

   x_velocity  y_velocity
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000  405.304839
0    0.000000    7.550217
0    0.000000   16.024052
0    0.000000   13.286987
0   18.287748   12.721911
0   25.179554   11.802916
0   24.626463    8.691693
0   25.800519    3.440069
0   25.215621    0.982427
0   25.605325    0.320067
0   25.367808    0.618727
0   25.815873    0.921995
0   25.919069    0.000000
0   24.582607    0.296176
0   25.817632    0.000000
0   25.080756    0.597161
0   24.280749    0.578113
0   25.159299    0.585100
0   24.378270    0.000000
0   23.267625    0.553991
0   21.761027    0.494569
0   23.621062    0.268421
0   23.544615    0.267552
0   23.071804    0.530386
0   22.758199    0.523177
0   29.886085    0.695025
0   77.231991   22.818543
0    0.000000    0.000000
0    0.000000    0.000000
0    0.000000    0.000000
0    0.00000